In [2]:
from utils import measure_accuracy, read_jsonl

# Run script to generate more samples

python scripts/generate_improved_from_judgements.py \
    --judgement-file /Users/ryanarman/code/lab/banking77/notebooks/data/evaluation_results_v2.jsonl \
    --project-id 1 \
    --output-file /Users/ryanarman/code/lab/banking77/notebooks/data/synth_v2.jsonl

## Merge and add system prompt and filter

python add_system_prompt.py /Users/ryanarman/code/lab/banking77/notebooks/data/synth_v2.jsonl /Users/ryanarman/code/lab/banking77/notebooks/data/banking77_train_improved_fixed.jsonl

In [ ]:
improved_train_path = "/Users/ryanarman/code/lab/banking77/notebooks/data/banking77_train_improved_fixed.jsonl"
improved_train_data = read_jsonl(improved_train_path)

# for e in improved_train_data:
#     # response = e['messages'][-1]['content']
#     print(e)
#     break

class_ids = [int(e['messages'][-1]['content']) for e in improved_train_data]
class_ids_set = set(class_ids)
print(f"min: {min(class_ids_set)}, max: {max(class_ids_set)}")

min: 0, max: 76


# Train the model

## Resume training from a checkpoint

To resume training from a previous checkpoint (e.g., `output/banking77_qwen3_4b_full_3009`), add the checkpoint path as the 8th argument:

```bash
./submit_training_rsync.sh \
data/banking77_train_improved_fixed.jsonl data/banking77_val.jsonl \
configs/4b_instruct_full.yaml \
banking77_qwen3_4b_full_improve \
ryan@exun \
banking77-qwen3-4b \
"" \
output/banking77_qwen3_4b_full_3009
```

**Note:** 
- The checkpoint path should be relative to the cluster's base directory (`/home/ryan/code/oumi/lab/banking77/notebooks/`)
- If you want to use default values for wandb_project and wandb_entity but specify a checkpoint, pass `""` for those arguments
- The checkpoint can be from either LoRA or full fine-tuning - oumi will automatically detect the format

## Full Fine-Tuning (FFT)

```bash
./submit_training_rsync.sh \
data/banking77_train_improved_fixed.jsonl data/banking77_val.jsonl \
configs/4b_instruct_full.yaml \
full_training_improved \
ryan@exun \
banking77-qwen3-4b
```

**Note:** Full fine-tuning uses FSDP (Fully Sharded Data Parallel) and requires more memory than LoRA. The config uses:
- `per_device_train_batch_size: 2`
- `gradient_accumulation_steps: 2`
- FSDP with `HYBRID_SHARD` strategy

## LoRA Fine-Tuning (Alternative)

```bash
./submit_training_rsync.sh \
data/banking77_train_improved_fixed.jsonl data/banking77_val.jsonl \
configs/qwen4b_train_lora.yaml \
my_training \
ryan@exun \
banking77-qwen3-4b
```

./submit_training_rsync.sh \
data/banking77_train_improved_fixed.jsonl \
data/banking77_val.jsonl \
configs/4b_instruct_full_1_epoch_mix_data.yaml \
full_training_improved \
ryan@exun \
banking77-qwen3-4b \
"" \
"" \
data/banking77_train.jsonl



/home/ryan/code/oumi/lab/banking77/notebooks/output/my_training_3014

# Run inference on trained model checkpoint

## Run inference


  
cd notebooks/scripts
./submit_inference_rsync.sh \
    data/banking77_test.jsonl \
    configs/4b_instruct_vllm_infer.yaml \
    improved_train_results_v2 \
    output/full_training_improved_3086  \
    ryan@exun


banking77_qwen3_4b_full_improve_3076 -> only improve batch accuracy: 69.12%, 9 rows with wrong output (not an id but also some text output) (inference improved_train_results_v2_3084)
banking77_qwen3_4b_full_improve_3077 -> only improve batch accuracy: 70.45%, 20 rows with wrong output (inference improved_train_results_v2_3085)
full_training_improved_3078 -> mixed data, first_exhausted (Accuracy: 85.32%) (inference improved_train_results_v2_3083)

full_training_improved_3086 -> mixed data, all_exhausted (Accuracy: 93.51%), val_accuracy:98.11% (inference improved_train_results_v2_3087)

run infernce on valdiation set
cd notebooks/scripts
./submit_inference_rsync.sh \
    data/banking77_val_no_label.jsonl \
    configs/4b_instruct_vllm_infer.yaml \
    improved_train_results_v2 \
    output/full_training_improved_3086  \
    ryan@exun


Val accuracy on best check point (banking77_qwen3_4b_full_3065): 91.60%, val accuracy on imporved model (full_training_improved_3086): 93.30%

In [8]:
inference_file = '/Users/ryanarman/code/lab/banking77/notebooks/data/improved_train_results_v2_3087.jsonl'
inference_data = read_jsonl(inference_file)
accuracy, correct, total, errors, incorrect_list = measure_accuracy(inference_data)


idx: 1047, gt_label: 6, inference_class_label: 1, content: 1–3 business days
Accuracy: 93.51% (2880/3080)


In [ ]:
inference_data[0]

{'conversation_id': '3aea8a2a-e323-514e-ba81-d721002ade39',
 'messages': [{'content': 'You are a banking intent classifier. Classify the user\'s query into one of  77 banking intents (output is a single integer ID).\n\nIDs:\n\n0: activate_my_card\n1: age_limit\n2: apple_pay_or_google_pay\n3: atm_support\n4: automatic_top_up\n5: balance_not_updated_after_bank_transfer\n6: balance_not_updated_after_cheque_or_cash_deposit\n7: beneficiary_not_allowed\n8: cancel_transfer\n9: card_about_to_expire\n10: card_acceptance\n11: card_arrival\n12: card_delivery_estimate\n13: card_linking\n14: card_not_working\n15: card_payment_fee_charged\n16: card_payment_not_recognised\n17: card_payment_wrong_exchange_rate\n18: card_swallowed\n19: cash_withdrawal_charge\n20: cash_withdrawal_not_recognised\n21: change_pin\n22: compromised_card\n23: contactless_not_working\n24: country_support\n25: declined_card_payment\n26: declined_cash_withdrawal\n27: declined_transfer\n28: direct_debit_payment_not_recognised\n29